In [1]:
import pandas as pd
import seaborn as sns
import mysql.connector
import matplotlib.pyplot as plt

In [2]:
# SQL Connection 

db = mysql.connector.connect( host ="localhost",
                             username="root",
                             password="root",
                             database="ecommerce")
cur = db.cursor()

In [3]:
# All unique cities where custoners are present
query = """ select distinct customer_city from customers"""
cur.execute(query)

data = cur.fetchall()

#data

In [4]:
# count of orders placed in 2017

query = """ select count(*) as orders_placed_in_2017 from orders where year(order_purchase_timestamp) = 2017 """

cur.execute(query)
data = cur.fetchall()

#data

In [5]:
# Find the total sales per category
query = """SELECT 
    products.product_category AS category,
    round(SUM(payments.payment_value),2) AS Sales
FROM 
    products
JOIN 
    order_items ON products.product_id = order_items.product_id
JOIN 
    payments ON order_items.order_id = payments.order_id
GROUP BY 
    products.product_category;
"""

cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data,columns = ["Category","Sales"])

df

,Category,Sales
0,perfumery,506738.66
1,Furniture Decoration,1430176.39
2,telephony,486882.05
3,bed table bath,1712553.67
4,automotive,852294.33
...,...,...
69,cds music dvds,1199.43
70,La Cuisine,2913.53
71,Fashion Children's Clothing,785.67
72,PC Gamer,2174.43


In [6]:
# Calculate the percentage of orders that were paid in installments
query = """ SELECT 
    (SUM(CASE WHEN payment_installments > 1 THEN 1 ELSE 0 END) * 100.0) / COUNT(*) AS percentage
FROM 
    ecommerce.payments;
"""

cur.execute(query)
data = cur.fetchall()

data[0]

(Decimal('49.41763'),)

In [7]:
# Count the number of customers from each state 

query = """ SELECT customer_state as States, count(customer_id) as No_of_customers from customers group by customer_state
"""

cur.execute(query)
data = cur.fetchall()

df = pd.DataFrame(data, columns = ["State","No of Customers"])
df = df.sort_values( by = "No of Customers" , ascending = False)
df

,State,No of Customers
0,SP,41746
4,RJ,12852
2,MG,11635
5,RS,5466
3,PR,5045
1,SC,3637
9,BA,3380
13,DF,2140
8,ES,2033
7,GO,2020


In [8]:
#  number of orders per month in 2018
# Define the query
query = """
SELECT 
   monthname(order_purchase_timestamp) AS Month,
    COUNT(*) AS no_of_orders
FROM 
    ecommerce.orders
WHERE 
    YEAR(order_purchase_timestamp) = 2018
GROUP BY 
    Month 
"""

# Execute the query
cur.execute(query)

# Fetch all results
data = cur.fetchall()
data 
df = pd.DataFrame(data, columns = ["Month", "No of Orders"])
df = df.sort_values(by = "Month")

df

,Month,No of Orders
7,April,6939
1,August,6512
2,February,6728
5,January,7269
0,July,6292
3,June,6167
4,March,7211
6,May,6873
9,October,4
8,September,16


In [11]:
# average number of products per order, grouped by customer city
query = """
with count_per_orders as (select orders.customer_id, count(order_items.order_id) as orders from order_items 
join orders on order_items.order_id = orders.order_id 
group by order_items.order_id, orders.customer_id)

select customers.customer_city as City ,  round(avg( count_per_orders.orders),2) as Avg_orders from customers 
join count_per_orders on customers.customer_id = count_per_orders.customer_id 
group by customers.customer_city;

"""

# Execute the query
cur.execute(query)

# Fetch all results
data = cur.fetchall()
data 

[('sao paulo', Decimal('1.16')),
 ('sao jose dos campos', Decimal('1.14')),
 ('porto alegre', Decimal('1.17')),
 ('indaial', Decimal('1.12')),
 ('treze tilias', Decimal('1.27')),
 ('rio de janeiro', Decimal('1.15')),
 ('mario campos', Decimal('1.33')),
 ('guariba', Decimal('1.00')),
 ('cuiaba', Decimal('1.20')),
 ('franca', Decimal('1.25')),
 ('tocos', Decimal('1.00')),
 ('januaria', Decimal('1.18')),
 ('campinas', Decimal('1.16')),
 ('embu-guacu', Decimal('1.24')),
 ('cerquilho', Decimal('1.17')),
 ('belo horizonte', Decimal('1.14')),
 ('taubate', Decimal('1.13')),
 ('arcos', Decimal('1.12')),
 ('monte mor', Decimal('1.08')),
 ('vianopolis', Decimal('1.33')),
 ('cacapava', Decimal('1.16')),
 ('sao jose da tapera', Decimal('1.00')),
 ('belford roxo', Decimal('1.09')),
 ('pedro leopoldo', Decimal('1.25')),
 ('arroio do sal', Decimal('1.29')),
 ('sao bernardo do campo', Decimal('1.14')),
 ('brasilia', Decimal('1.13')),
 ('ribeirao pires', Decimal('1.12')),
 ('contagem', Decimal('1.14')),